Import modules

In [ ]:
import numpy as np
import os
import scipy
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from T import *
from trans import *
from readx import *
from scipy.optimize import fmin, curve_fit

Initial Data

In [ ]:
ref = open(r"C:\Users\Rohith K M\OneDrive - Indian Institute of Technology Guwahati\Lab Data THz\Suchandra\28-05-25 LCO\air_5mm.picotd")
sam = open(r"C:\Users\Rohith K M\OneDrive - Indian Institute of Technology Guwahati\Lab Data THz\Suchandra\28-05-25 LCO\23.picotd")

case      = 'BBIO_60K.txt'
subcase   = [case,'0']

save_path = r"C:\Users\Rohith K M\OneDrive - Indian Institute of Technology Guwahati\Lab Data THz\Suchandra\28-05-25 LCO\Results\Refractive index - home built"

t1 , t2 = 0 , 699      #For reference

t3 , t4 = 0 , 699      #For sample

f1 , f2 = 0.2 , 1.0         #Frequency range

n1  = complex(1.0,0.0)        
n2  = [3.7,0.25]               #initial n,k (Take from cell below)
n3  = complex(1.0,0.0)     #substrate

L = 880e-6                 #sample thickness

Initial n,k

In [ ]:
%matplotlib qt
t,dphi,omega,data,dphi_1,time_shift,amp_ratio = trans(t1,t2,f1,f2,ref,sam,t3,t4)

n_guess = np.zeros(len(omega))
k_guess = np.zeros(len(omega))
n_guess += ( scipy.constants.c/ L) * time_shift*1e-12 + 1.0
k_guess += -scipy.constants.c * np.log(amp_ratio) / (L * omega)

freq = omega/(2*np.pi)
freq_THz = freq/1e12

plt.figure(4)
plt.subplot(1,2,1)
plt.plot(freq_THz,n_guess,label='n_guess')
plt.subplot(1,2,2)
plt.plot(freq_THz,k_guess,label='k_guess')
plt.show()

Objective function

In [ ]:
# def min_func(x, L, omega, t, t_theo, dphi, k, dM, dPhi, n1, n3):
#     t_theo[k] = T(n1, x, n3, omega, L)
#     M = np.log(np.abs(t_theo[k]) / np.abs(t[k]))
    
#     # Safeguard slicing
#     slice_data = t_theo[-1:k-1:-1]
#     if slice_data.size > 0:
#         Phi = np.unwrap(np.angle(slice_data))[-1] - dphi[k]
#     else:
#         Phi = 0  # Handle empty slice

#     dM[k] = M
#     dPhi[k] = Phi
#     xi = 1.0
#     return np.linalg.norm(dM) + xi * np.linalg.norm(dPhi)

def min_func(x, L, omega, t, t_theo, dphi, k, dM, dPhi, n1, n3):
    
    t_theo[k] = T(n1, x, n3, omega, L)
    u = dphi[:k + 1]
    u_approx = np.unwrap(np.angle(t_theo[:k + 1]))
    error = np.abs(np.abs(t_theo[k]) - np.abs(t[k])) + np.abs(u[k] - u_approx[k])
    return error

Function calls and optimization

In [ ]:
dphi=1.0*dphi
n_r = np.zeros(len(omega),dtype=float)
n_i = np.zeros(len(omega),dtype=float)
t_theo = np.zeros(len(omega),dtype = complex)
dM = np.zeros(len(omega),dtype=float)
dPhi = np.zeros(len(omega),dtype=float)
epsilon = np.zeros(len(omega),dtype=complex)
sigma = np.zeros(len(omega),dtype = complex)

k=0
for j in omega:
    print(j,k)
    res = fmin (min_func,
                           x0=n2,
                           xtol = 1e-8,
                           args=(L,j,t,t_theo,dphi,k,dM,dPhi,n1,n3),
                           disp=True
                           )

    n2   = [res[0],res[1]]
    
    n_r[k] = res[0]
    n_i[k] = res[1]
    epsilon[k] = complex(n_r[k],n_i[k])**2
    sigma[k]  = j*scipy.constants.epsilon_0*(np.imag(epsilon[k]) - complex(0,1)*(np.real(epsilon[k])-1.0))
    k+=1

freq = omega/(2*np.pi)
freq_THz = freq/1e12

Plots

In [ ]:
%matplotlib qt
ekk = datetime.datetime.now()
n_initial = subcase
d,m,y,hr,mi = ekk.day,ekk.month,ekk.year,ekk.hour,ekk.minute
day_time = str(d)+'-'+str(m)+'-'+str(y)+' '+str(hr)+':'+str(mi)
mat_info = 'n3='+str(n3)+' L2='+str("%0.3f" % (L*1e6))+'um  '
font_xaxis = 7.5
font_yaxis = 7.5
font_title = 8
font_legend = 8

plt.figure(0)
plt.subplot(2,2,1)
plt.plot(freq_THz,n_r,label='Real')
plt.plot(freq_THz,n_i,label='Imag')
plt.xlabel('Frequency ['+str(f1)+' THz-'+str(f2)+' THz]',fontsize = font_xaxis)
plt.ylabel('Complex refractive index',fontsize = font_yaxis)
plt.legend(fontsize = font_legend)
plt.axhline(y=0,linestyle = '--',color='darkgray',linewidth = 1)
#plt.savefig('nk.png',dpi=300)
#plt.show()
#plt.close()

plt.subplot(2,2,2)
plt.plot(freq_THz,np.abs(t),label='Exp')
plt.plot(freq_THz,np.abs(t_theo),label='Theory')
plt.xlabel('Frequency (THz)',fontsize = font_xaxis)
plt.ylabel('Mag(t)',fontsize = font_yaxis)
plt.ylim(0,1.5)
plt.legend(fontsize = font_legend)
plt.axhline(y=1.0,linestyle = '--',color='darkgray',linewidth = 1)
#plt.savefig('t_mag.png',dpi=300)
#plt.show()
#plt.close()

name1 = 'all_'+str(n_initial[0])+'_'+str(n_initial[1])+'_'+str(t1)+'_'+str(t2)+'.png'
plt.subplot(2,2,3)
plt.plot(freq_THz,np.angle(t),label='Exp')
plt.plot(freq_THz,np.angle(t_theo),label='Theory')
plt.plot(freq_THz,dphi)
plt.xlabel('Frequency (THz)',fontsize = font_xaxis)
plt.ylabel('Phase''[$\phi_{sam}-\phi_{ref}$]',fontsize = font_yaxis)
plt.legend(fontsize = font_legend)

plt.subplot(2,2,4)
plt.plot(data.time_ref,data.v_ref,label='Ref')
plt.plot(data.time_sam,data.v_sam,label='Sam')
plt.xlabel('Time ['+str(np.min(data.time_ref))+'ps - '+str(np.max(data.time_ref))+'ps]' ,fontsize = font_xaxis)
plt.ylabel('Voltage',fontsize = font_yaxis)
plt.legend(fontsize = font_legend)
plt.axhline(y=0,linestyle = '--',color='darkgray',linewidth = 1)

plt.suptitle(mat_info+day_time,fontsize = font_title)
plt.tight_layout()

s_path = os.path.join(save_path, name1)
plt.savefig(s_path,dpi=300)
plt.show()

name2 = 'nk_'+str(n_initial[0])+'_'+str(n_initial[1])+'_'+str(t1)+'_'+str(t2)+'.png'
plt.figure(1)
plt.plot(freq_THz,n_r,label='Real')
plt.plot(freq_THz,n_i,label='Imag')
plt.xlabel('Frequency ['+str(f1)+' THz-'+str(f2)+' THz]',fontsize = font_xaxis)
plt.ylabel('Complex refractive index',fontsize = font_yaxis)
plt.legend(fontsize = font_legend)
plt.axhline(y=0,linestyle = '--',color='darkgray',linewidth = 1)

s_path = os.path.join(save_path, name2)
plt.savefig(s_path,dpi=300)
plt.show()

name3 = 'eps_'+str(n_initial[0])+'_'+str(n_initial[1])+'_'+str(t1)+'_'+str(t2)+'.png'
plt.figure(2)
plt.plot(freq_THz,np.real(epsilon),label='Real')
plt.plot(freq_THz,np.imag(epsilon),label='Imag')
plt.legend()
plt.axhline(y=0,linestyle = '--',color='darkgray',linewidth = 1)
plt.ylabel('Complex permitivitty')
plt.xlabel('THz')

s_path = os.path.join(save_path, name3)
plt.savefig(s_path,dpi=300)
plt.show()

plt.figure(3)
plt.plot(freq_THz,np.real(sigma),label='Real')
plt.plot(freq_THz,np.imag(sigma),label='Imag')
plt.legend()
plt.axhline(y=0,linestyle = '--',color='darkgray',linewidth = 1)
plt.ylabel('Complex conductivity')
plt.xlabel('THz')
s_path = os.path.join(save_path, 'sigma_'+case+'.png')
plt.savefig(s_path,dpi=300)
plt.show()

WRITE DATA

In [ ]:
t_data = np.array([freq_THz,np.real(t),np.imag(t)]).T
nk_data = np.array([freq_THz,n_r,n_i]).T
eps_data = np.array([freq_THz,np.real(epsilon),np.imag(epsilon)]).T
sigma_data = np.array([freq_THz,np.real(sigma),np.imag(sigma)]).T
v_ref_data = np.array([data.time_ref,data.v_ref]).T
v_sam_data = np.array([data.time_sam,data.v_sam]).T

s_path = os.path.join(save_path, 't_'+case)
np.savetxt(s_path, t_data,delimiter='\t', newline='\n')

s_path = os.path.join(save_path, 'nk_'+case)
np.savetxt(s_path, nk_data,delimiter='\t', newline='\n')

s_path = os.path.join(save_path, 'eps_'+case)
np.savetxt(s_path, eps_data,delimiter='\t', newline='\n')

s_path = os.path.join(save_path, 'sigma_'+case)
np.savetxt(s_path, sigma_data,delimiter='\t', newline='\n')

s_path = os.path.join(save_path, 'td_ref_'+case)
np.savetxt(s_path, v_ref_data,delimiter='\t', newline='\n')

s_path = os.path.join(save_path, 'td_sam_'+case)
np.savetxt(s_path, v_sam_data,delimiter='\t', newline='\n')
